In [ ]:
!pip install transformers
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [ ]:
dataset

In [ ]:
dataset["train"][10]

In [ ]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(dataset["train"])

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [ ]:
# 完整的超参数配置
print(training_args)

In [ ]:
!pip install evaluate
!pip install scikit-learn

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [ ]:
!pip uninstall -y deepspeed

In [9]:
# !pip install --upgrade transformers deepspeed
# !echo xuwei | sudo apt-get install mpich 
!pip install mpi4py
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
import evaluate

#model_dir = "/data/models/bert-base-cased-finetuned"
model_dir = "models/bert-base-cased"
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  fp16=True,
                                  save_steps=1000,
                                  num_train_epochs=1,
                                  deepspeed="deepspeed_config.json",
                                  logging_steps=1000)



dataset = load_dataset("yelp_review_full")

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

metric = evaluate.load("accuracy")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#准备训练数据集和测试数据集
# train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
#开始训练
trainer.train()
trainer.save_model(f"{model_dir}/finetuned-trainer")
trainer.save_state()

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2024-01-18 18:42:15,170] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter cpu_offload is deprecated use offload_optimizer instead


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.339844,0.427000


In [ ]:
!nvidia-smi
! sudo kill -9 1778429      

In [ ]:
!pip list | grep torch
!echo y | pip uninstall torch
!pip install torch==2.0.1
!pip list | grep torch

In [ ]:
import getpass
import os

password = getpass.getpass()
command = "sudo -S apt-get update" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))

! sudo apt-get update
! sudo apt-get install libopenmpi-dev

In [ ]:
 # 为加速的情况[ 781/243750 03:25 < 17:46:44, 3.80 it/s, Epoch 0.01/3]
# 尝试使用加速
!pip install deepspeed
!pip install transformers
!pip install datasets
!pip install mpi4py

from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from datasets import load_dataset

model_dir = "models/bert-base-cased"
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  deepspeed="./deepspeed_config.json",
                                  fp16=True, 
                                  logging_steps=100)



dataset = load_dataset("yelp_review_full")

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#准备训练数据集和测试数据集
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
#开始训练
trainer.train()
trainer.save_model(f"{model_dir}/finetuned-trainer")
trainer.save_state()